In [17]:
import dgl
import pandas as pd
import torch

okved_g = pd.read_pickle('../data/okved_graph.pickle')
okved_data = pd.read_csv('../data/okved_2014_w_sections.csv', index_col=0)

In [2]:
# Словари id-шники - коды и коды - id-шники
id_to_code = okved_data['native_code'].to_dict()
id_to_code[0] = '0'
code_to_id = {v: u for u, v in id_to_code.items()}

In [3]:
sample_codes = okved_data[okved_data['native_code'].str.slice(0, 2) == '59'].index.values
g_sample = okved_g.subgraph(sample_codes, store_ids=True)
g_sample = g_sample.edge_subgraph(g_sample.edata['type'] == 1, relabel_nodes=False)
g_sample.edges()

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 5, 5, 5, 6, 7, 8, 9, 9]),
 tensor([9, 1, 2, 3, 4, 0, 0, 0, 0, 9, 6, 7, 8, 5, 5, 5, 0, 5]))

In [4]:
sample_id_to_code = list(map(lambda x: id_to_code[x], g_sample.ndata[dgl.NID].numpy()))
sample_id_to_code = dict(zip(range(len(sample_id_to_code)), sample_id_to_code))

graph = [g_sample.edges()[0].numpy(), g_sample.edges()[1].numpy()]
graph[0] = list(map(lambda x: sample_id_to_code[x], graph[0]))
graph[1] = list(map(lambda x: sample_id_to_code[x], graph[1]))
graph = pd.DataFrame(graph).T.sort_values(0).reset_index(drop=True)
graph = graph.loc[[0, 1, 2, 4, 5, 6, 12, 13, 14]]

In [9]:
g_sample.edata

{'weight': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'type': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'train_mask': tensor([ True, False,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True]), 'norm': tensor([0.2582, 0.4472, 0.4472, 0.4472, 0.4472, 0.4472, 0.4472, 0.4472, 0.4472,
        0.2887, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.2582, 0.2887]), '_ID': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])}

In [5]:
graph

,0,1
0,59,59.2
1,59,59.1
2,59.1,59.14
4,59.1,59.12
5,59.1,59.11
6,59.1,59.13
12,59.2,59.20.1
13,59.2,59.20.2
14,59.2,59.20.3


In [6]:
import os
os.environ["PATH"] += os.pathsep + r'C:/Program Files (x86)/Graphviz/bin/'

In [7]:
import graphviz

G = graphviz.Digraph('G', filename='59_sample')
for _, (u, v) in graph.iterrows():
    G.edge(u, v)
G.edge_attr.update(arrowsize='0.5')
G.view()

'59_sample.pdf'

In [12]:
okved_g

Graph(num_nodes=2637, num_edges=438730,
      ndata_schemes={'feat': Scheme(shape=(312,), dtype=torch.float64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64), 'type': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'norm': Scheme(shape=(), dtype=torch.float32)})

In [27]:
torch.unique(okved_g.edata['type'], return_counts=True)

(tensor([0, 1, 2]), tensor([175868,   5272, 257590]))

In [31]:
info_table = pd.DataFrame({' ': [2637, 175868, 5272, 257590, 438730]},
                          index=['Количество узлов', 'Связей типа 0', 'Связей типа 1', 'Связей типа 2', 'Общее количество связей'])
print(info_table.style.to_latex())

\begin{tabular}{lr}
{} & { } \\
Количество узлов & 2637 \\
Связей типа 0 & 175868 \\
Связей типа 1 & 5272 \\
Связей типа 2 & 257590 \\
Общее количество связей & 438730 \\
\end{tabular}

